<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectifs" data-toc-modified-id="Objectifs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectifs</a></span></li><li><span><a href="#Dev" data-toc-modified-id="Dev-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dev</a></span><ul class="toc-item"><li><span><a href="#Lecture-d'un-fichier" data-toc-modified-id="Lecture-d'un-fichier-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Lecture d'un fichier</a></span></li><li><span><a href="#Concatenation-des-fichiers-et-traitement" data-toc-modified-id="Concatenation-des-fichiers-et-traitement-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Concatenation des fichiers et traitement</a></span><ul class="toc-item"><li><span><a href="#Concatatenation-des-fichiers-et-retraitement-simple" data-toc-modified-id="Concatatenation-des-fichiers-et-retraitement-simple-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Concatatenation des fichiers et retraitement simple</a></span></li><li><span><a href="#Vérification-des-données-(doublons)" data-toc-modified-id="Vérification-des-données-(doublons)-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Vérification des données (doublons)</a></span></li></ul></li><li><span><a href="#Ajout-de-features-+-resampling-des-données-sur-10-min" data-toc-modified-id="Ajout-de-features-+-resampling-des-données-sur-10-min-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Ajout de features + resampling des données sur 10 min</a></span><ul class="toc-item"><li><span><a href="#Ajout-des-features-avant-resampling" data-toc-modified-id="Ajout-des-features-avant-resampling-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Ajout des features avant resampling</a></span></li><li><span><a href="#Resampling" data-toc-modified-id="Resampling-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Resampling</a></span></li><li><span><a href="#Fonction-de-création" data-toc-modified-id="Fonction-de-création-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>Fonction de création</a></span></li></ul></li></ul></li><li><span><a href="#Industrialisation" data-toc-modified-id="Industrialisation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Industrialisation</a></span></li><li><span><a href="#Analyse" data-toc-modified-id="Analyse-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Analyse</a></span><ul class="toc-item"><li><span><a href="#Absence-de-data-dans-la-Time-Series" data-toc-modified-id="Absence-de-data-dans-la-Time-Series-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Absence de data dans la Time Series</a></span></li><li><span><a href="#Analyse-des-déconnexion-des-stations" data-toc-modified-id="Analyse-des-déconnexion-des-stations-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Analyse des déconnexion des stations</a></span></li></ul></li></ul></div>

In [5]:
import glob as glob

import pandas as pd
import polars as pl

from vcub_keeper.config import ROOT_DATA_CLEAN, ROOT_DATA_RAW
from vcub_keeper.transform.features_factory import get_transactions_all, get_transactions_in, get_transactions_out

pd.set_option("display.max_rows", 700)

%load_ext autoreload
%autoreload 2

## Objectifs

- Lecture des données des vcub de Bordeaux (raw).
- Concaténation de ces fichiers en une seul.
- Data Check.
- Ajout de features (transactions).
- Resample data (https://github.com/armgilles/vcub_keeper/issues/11)
- Export du fichier dans /data/clean/.

Gihub : https://github.com/armgilles/vcub_keeper/issues/17

## Dev

### Lecture d'un fichier

In [5]:
file_name = "bordeaux-2019.csv"

column_dtypes = {"id": "uint8", "status": "category", "available_stands": "uint8", "available_bikes": "uint8"}
activite = pd.read_csv(ROOT_DATA_RAW + file_name, parse_dates=["timestamp"], dtype=column_dtypes)

In [6]:
activite.head()

,id,status,available_stands,available_bikes,timestamp
0,128,open,0,16,2019-08-20 04:38:01
1,24,open,7,9,2019-12-17 17:17:03
2,244,open,30,0,2019-06-13 01:10:01
3,17,open,1,13,2019-03-22 00:03:02
4,167,open,17,3,2019-07-07 10:58:02


In [4]:
activite.tail()

,id,status,available_stands,available_bikes,timestamp
12527714,50,open,13,1,2019-04-01 15:37:02
12527715,125,open,18,12,2019-12-11 23:13:02
12527716,58,open,4,14,2019-12-02 16:18:02
12527717,125,closed,29,1,2019-03-31 19:48:02
12527718,147,open,17,12,2019-12-26 14:13:13


In [5]:
# Vérification d'une données pour l'id 39
# activite = activite.sort_values(['id', 'timestamp'])
activite[(activite["id"] == 39) & (activite["timestamp"] >= "2019-07-19 16:10:00")].head(20)

,id,status,available_stands,available_bikes,timestamp
642,39,open,40,0,2019-09-05 22:52:02
840,39,open,26,13,2019-08-15 17:19:02
924,39,open,27,13,2019-12-28 02:48:14
977,39,open,25,15,2019-09-26 18:15:02
1046,39,open,39,0,2019-09-17 18:23:02
1213,39,open,38,2,2019-10-17 22:24:02
2488,39,open,28,11,2019-08-28 17:27:02
3146,39,open,26,12,2019-09-16 12:22:02
3210,39,open,30,10,2019-11-17 11:28:02
3671,39,open,27,13,2019-11-17 18:02:02


In [6]:
activite.dtypes

id                           uint8
status                    category
available_stands             uint8
available_bikes              uint8
timestamp           datetime64[ns]
dtype: object

In [7]:
activite.shape

(12527719, 5)

In [8]:
activite.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12527719 entries, 0 to 12527718
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                uint8         
 1   status            category      
 2   available_stands  uint8         
 3   available_bikes   uint8         
 4   timestamp         datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint8(3)
memory usage: 143.4 MB


In [9]:
activite.memory_usage(deep=True)

Index                     128
id                   12527719
status               12527923
available_stands     12527719
available_bikes      12527719
timestamp           100221752
dtype: int64

### Concatenation des fichiers et traitement

#### Concatatenation des fichiers et retraitement simple

In [10]:
activite_full = pd.DataFrame()

for file_path in glob.glob(ROOT_DATA_RAW + "bordeaux-*.csv"):
    file_name = file_path.split("/")[-1]
    print(file_name)

    # Lecture du fichier
    column_dtypes = {"id": "uint8", "status": "category", "available_stands": "uint8", "available_bikes": "uint8"}
    activite_temp = pd.read_csv(ROOT_DATA_RAW + file_name, parse_dates=["timestamp"], dtype=column_dtypes)
    print(activite_temp.shape)

    # Concact
    activite_full = pd.concat([activite_full, activite_temp])

# Working on concatenate file

# Status mapping
status_dict = {"open": 1, "closed": 0}
activite_full["status"] = activite_full["status"].map(status_dict)
activite_full["status"] = activite_full["status"].astype("uint8")

# Naming
activite_full.rename(columns={"id": "station_id"}, inplace=True)
activite_full.rename(columns={"timestamp": "date"}, inplace=True)

# Sorting DataFrame on station_id & date
activite_full = activite_full.sort_values(["station_id", "date"], ascending=[1, 1])

# Reset index
activite_full = activite_full.reset_index(drop=True)

bordeaux-2018.csv
(1118425, 5)
bordeaux-2019.csv
(12527719, 5)
bordeaux-2020.csv
(11888595, 5)


In [11]:
activite_full.head()

,station_id,status,available_stands,available_bikes,date
0,1,1,28,4,2018-12-01 00:04:02
1,1,1,28,4,2018-12-01 00:09:03
2,1,1,28,4,2018-12-01 00:14:03
3,1,1,28,4,2018-12-01 00:19:03
4,1,1,28,4,2018-12-01 00:24:03


In [12]:
activite_full.dtypes

station_id                   uint8
status                       uint8
available_stands             uint8
available_bikes              uint8
date                datetime64[ns]
dtype: object

In [13]:
activite_full.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25534739 entries, 0 to 25534738
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   station_id        uint8         
 1   status            uint8         
 2   available_stands  uint8         
 3   available_bikes   uint8         
 4   date              datetime64[ns]
dtypes: datetime64[ns](1), uint8(4)
memory usage: 292.2 MB


In [14]:
activite_full.memory_usage(deep=True)

Index                     128
station_id           25534739
status               25534739
available_stands     25534739
available_bikes      25534739
date                204277912
dtype: int64

#### Vérification des données (doublons)

In [15]:
# Pas de doublon station_id / date
grp = activite_full.groupby(["station_id", "date"])["available_stands"].count()

In [16]:
grp[grp > 1]

station_id  date               
26          2020-02-06 04:16:13    2
Name: available_stands, dtype: int64

In [17]:
activite_full[(activite_full["station_id"] == 26) & (activite_full["date"] == "2020-02-06 04:16:13")]

,station_id,status,available_stands,available_bikes,date
3462142,26,1,10,5,2020-02-06 04:16:13
3462143,26,1,0,0,2020-02-06 04:16:13


Il existe des doublons (un seul)

In [18]:
activite_full = activite_full.drop_duplicates(subset=["station_id", "date"]).reset_index(drop=True)

In [19]:
activite_full.shape

(25534738, 5)

### Ajout de features + resampling des données sur 10 min

#### Ajout des features avant resampling

In [20]:
activite_full.head()

,station_id,status,available_stands,available_bikes,date
0,1,1,28,4,2018-12-01 00:04:02
1,1,1,28,4,2018-12-01 00:09:03
2,1,1,28,4,2018-12-01 00:14:03
3,1,1,28,4,2018-12-01 00:19:03
4,1,1,28,4,2018-12-01 00:24:03


In [21]:
activite_full = get_transactions_in(activite_full)
activite_full = get_transactions_out(activite_full)
activite_full = get_transactions_all(activite_full)

In [22]:
activite_full["available_bikes_shift"] = activite_full.groupby("station_id")["available_bikes"].shift(1)
activite_full["available_bikes_shift"] = activite_full["available_bikes_shift"].fillna(activite_full["available_bikes"])

In [23]:
activite_full["transactions_in"] = activite_full["available_bikes"] - activite_full["available_bikes_shift"]

activite_full.loc[activite_full["transactions_in"] < 0, "transactions_in"] = 0

In [24]:
activite_full[(activite_full["station_id"] == 39) & (activite_full["date"] >= "2019-07-19 16:00:00")].head(20)

,station_id,status,available_stands,available_bikes,date,transactions_in,transactions_out,transactions_all,available_bikes_shift
5209692,39,1,0,40,2019-07-19 16:03:02,0.0,0.0,0.0,40.0
5209693,39,1,0,40,2019-07-19 16:08:02,0.0,0.0,0.0,40.0
5209694,39,1,0,39,2019-07-19 16:13:02,0.0,0.0,1.0,40.0
5209695,39,1,0,39,2019-07-19 16:19:01,0.0,0.0,0.0,39.0
5209696,39,1,0,39,2019-07-19 16:24:02,0.0,0.0,0.0,39.0
5209697,39,1,0,39,2019-07-19 16:29:01,0.0,0.0,0.0,39.0
5209698,39,1,0,39,2019-07-19 16:32:02,0.0,0.0,0.0,39.0
5209699,39,1,0,39,2019-07-19 16:37:02,0.0,0.0,0.0,39.0
5209700,39,1,0,39,2019-07-19 16:43:02,0.0,0.0,0.0,39.0
5209701,39,1,0,39,2019-07-19 16:48:02,0.0,0.0,0.0,39.0


In [25]:
activite_full.shape

(25534738, 9)

In [26]:
activite_full.head()

,station_id,status,available_stands,available_bikes,date,transactions_in,transactions_out,transactions_all,available_bikes_shift
0,1,1,28,4,2018-12-01 00:04:02,0.0,0.0,0.0,4.0
1,1,1,28,4,2018-12-01 00:09:03,0.0,0.0,0.0,4.0
2,1,1,28,4,2018-12-01 00:14:03,0.0,0.0,0.0,4.0
3,1,1,28,4,2018-12-01 00:19:03,0.0,0.0,0.0,4.0
4,1,1,28,4,2018-12-01 00:24:03,0.0,0.0,0.0,4.0


In [27]:
# Check de la données source à certains moments :

station_id = 15
date = "2020-07-20"
activite_full[(activite_full["station_id"] == station_id) & (activite_full["date"] >= date)].head(500)

,station_id,status,available_stands,available_bikes,date,transactions_in,transactions_out,transactions_all,available_bikes_shift
2079976,15,1,3,11,2020-07-20 00:02:13,0.0,0.0,0.0,11.0
2079977,15,1,3,11,2020-07-20 00:07:13,0.0,0.0,0.0,11.0
2079978,15,1,3,11,2020-07-20 00:12:13,0.0,0.0,0.0,11.0
2079979,15,1,3,11,2020-07-20 00:17:13,0.0,0.0,0.0,11.0
2079980,15,1,3,11,2020-07-20 00:22:13,0.0,0.0,0.0,11.0
2079981,15,1,3,11,2020-07-20 00:27:13,0.0,0.0,0.0,11.0
2079982,15,1,3,11,2020-07-20 00:32:13,0.0,0.0,0.0,11.0
2079983,15,1,3,11,2020-07-20 00:37:13,0.0,0.0,0.0,11.0
2079984,15,1,3,11,2020-07-20 00:42:13,0.0,0.0,0.0,11.0
2079985,15,1,3,11,2020-07-20 00:47:13,0.0,0.0,0.0,11.0


In [28]:
activite_full_copy = activite_full.copy()
# activite_full = activite_full_copy

#### Resampling

In [29]:
# On doit mettre la date en index
# cf Bug Pandas : https://github.com/pandas-dev/pandas/issues/33548
activite_full = activite_full.set_index("date")

In [30]:
activite_full.head()

,station_id,status,available_stands,available_bikes,transactions_in,transactions_out,transactions_all,available_bikes_shift
date,,,,,,,,
2018-12-01 00:04:02,1,1,28,4,0.0,0.0,0.0,4.0
2018-12-01 00:09:03,1,1,28,4,0.0,0.0,0.0,4.0
2018-12-01 00:14:03,1,1,28,4,0.0,0.0,0.0,4.0
2018-12-01 00:19:03,1,1,28,4,0.0,0.0,0.0,4.0
2018-12-01 00:24:03,1,1,28,4,0.0,0.0,0.0,4.0


In [31]:
activite_full_resample = (
    activite_full.groupby("station_id")
    .resample(
        "10T",
        label="right",
    )
    .agg(
        {
            "available_stands": "last",
            "available_bikes": "last",
            "status": "max",
            "transactions_in": "sum",
            "transactions_out": "sum",
            "transactions_all": "sum",
        }
    )
    .reset_index()
)

# Casting to save memory
# activite_full_resample['station_id'] = activite_full_resample['station_id'].astype('uint8')

In [32]:
activite_full = activite_full.reset_index()

In [33]:
activite_full_resample.dtypes

station_id                  uint64
date                datetime64[ns]
available_stands           float64
available_bikes            float64
status                     float64
transactions_in            float64
transactions_out           float64
transactions_all           float64
dtype: object

In [34]:
activite_full_resample.info(verbose=True, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16564512 entries, 0 to 16564511
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   station_id        uint64        
 1   date              datetime64[ns]
 2   available_stands  float64       
 3   available_bikes   float64       
 4   status            float64       
 5   transactions_in   float64       
 6   transactions_out  float64       
 7   transactions_all  float64       
dtypes: datetime64[ns](1), float64(6), uint64(1)
memory usage: 1011.0 MB


In [35]:
activite_full_resample.memory_usage(deep=True)

Index                     128
station_id          132516096
date                132516096
available_stands    132516096
available_bikes     132516096
status              132516096
transactions_in     132516096
transactions_out    132516096
transactions_all    132516096
dtype: int64

In [36]:
activite_full_resample.shape

(16564512, 8)

In [37]:
# TO KEEP
# La bonne config pour available_stands etc
# test = activite_full[activite_full.station_id == 1]
# activite_full_resample = \
#    test.resample('10T',
#                 on='date',
#                 label='right',
#                 #closed='right'
#                 #convention='end'
#                )['available_stands'].last()


In [38]:
activite_full_resample.head(8)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
0,1,2018-12-01 00:10:00,28.0,4.0,1.0,0.0,0.0,0.0
1,1,2018-12-01 00:20:00,28.0,4.0,1.0,0.0,0.0,0.0
2,1,2018-12-01 00:30:00,28.0,4.0,1.0,0.0,0.0,0.0
3,1,2018-12-01 00:40:00,27.0,5.0,1.0,1.0,0.0,1.0
4,1,2018-12-01 00:50:00,27.0,5.0,1.0,0.0,0.0,0.0
5,1,2018-12-01 01:00:00,28.0,4.0,1.0,0.0,1.0,1.0
6,1,2018-12-01 01:10:00,28.0,4.0,1.0,0.0,0.0,0.0
7,1,2018-12-01 01:20:00,28.0,4.0,1.0,0.0,0.0,0.0


In [39]:
activite_full[(activite_full["station_id"] == 1) & (activite_full.date < "2018-12-01 01:00:00")]

,date,station_id,status,available_stands,available_bikes,transactions_in,transactions_out,transactions_all,available_bikes_shift
0,2018-12-01 00:04:02,1,1,28,4,0.0,0.0,0.0,4.0
1,2018-12-01 00:09:03,1,1,28,4,0.0,0.0,0.0,4.0
2,2018-12-01 00:14:03,1,1,28,4,0.0,0.0,0.0,4.0
3,2018-12-01 00:19:03,1,1,28,4,0.0,0.0,0.0,4.0
4,2018-12-01 00:24:03,1,1,28,4,0.0,0.0,0.0,4.0
5,2018-12-01 00:29:04,1,1,28,4,0.0,0.0,0.0,4.0
6,2018-12-01 00:34:02,1,1,28,4,0.0,0.0,0.0,4.0
7,2018-12-01 00:39:02,1,1,27,5,1.0,0.0,1.0,4.0
8,2018-12-01 00:44:03,1,1,27,5,0.0,0.0,0.0,5.0
9,2018-12-01 00:49:05,1,1,27,5,0.0,0.0,0.0,5.0


In [40]:
activite_full_resample.tail()

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
16564507,251,2020-08-28 11:10:00,28.0,12.0,1.0,0.0,0.0,0.0
16564508,251,2020-08-28 11:20:00,28.0,12.0,1.0,0.0,0.0,0.0
16564509,251,2020-08-28 11:30:00,26.0,14.0,1.0,2.0,0.0,2.0
16564510,251,2020-08-28 11:40:00,26.0,14.0,1.0,0.0,0.0,0.0
16564511,251,2020-08-28 11:50:00,26.0,14.0,1.0,0.0,0.0,0.0


In [41]:
activite_full[(activite_full["station_id"] == 251) & (activite_full.date < "2020-08-28 11:50:00")].tail(9)

,date,station_id,status,available_stands,available_bikes,transactions_in,transactions_out,transactions_all,available_bikes_shift
25534729,2020-08-28 11:03:14,251,1,28,12,0.0,0.0,0.0,12.0
25534730,2020-08-28 11:08:14,251,1,28,12,0.0,0.0,0.0,12.0
25534731,2020-08-28 11:13:14,251,1,28,12,0.0,0.0,0.0,12.0
25534732,2020-08-28 11:18:14,251,1,28,12,0.0,0.0,0.0,12.0
25534733,2020-08-28 11:23:14,251,1,27,13,1.0,0.0,1.0,12.0
25534734,2020-08-28 11:27:14,251,1,26,14,1.0,0.0,1.0,13.0
25534735,2020-08-28 11:33:14,251,1,26,14,0.0,0.0,0.0,14.0
25534736,2020-08-28 11:38:14,251,1,26,14,0.0,0.0,0.0,14.0
25534737,2020-08-28 11:43:14,251,1,26,14,0.0,0.0,0.0,14.0


On regarde comment est aggrégé `status`

In [42]:
# activite_full[activite_full.status == 0].head(5) # 7866 / 8659
activite_full[7860:7870]

,date,station_id,status,available_stands,available_bikes,transactions_in,transactions_out,transactions_all,available_bikes_shift
7860,2019-01-07 14:09:03,1,1,8,24,0.0,2.0,2.0,26.0
7861,2019-01-07 14:14:03,1,1,10,22,0.0,2.0,2.0,24.0
7862,2019-01-07 14:19:03,1,1,10,22,0.0,0.0,0.0,22.0
7863,2019-01-07 14:24:03,1,1,9,23,1.0,0.0,1.0,22.0
7864,2019-01-07 14:29:03,1,1,9,23,0.0,0.0,0.0,23.0
7865,2019-01-07 14:34:03,1,1,10,22,0.0,1.0,1.0,23.0
7866,2019-01-07 14:39:03,1,0,11,21,0.0,1.0,1.0,22.0
7867,2019-01-07 14:44:03,1,0,11,21,0.0,0.0,0.0,21.0
7868,2019-01-07 14:49:02,1,1,10,22,1.0,0.0,1.0,21.0
7869,2019-01-07 14:54:03,1,1,10,22,0.0,0.0,0.0,22.0


In [43]:
activite_full_resample[
    (activite_full_resample["station_id"] == 1) & (activite_full_resample.date <= "2019-01-07 15:00:00")
].tail(6)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
5412,1,2019-01-07 14:10:00,8.0,24.0,1.0,0.0,3.0,3.0
5413,1,2019-01-07 14:20:00,10.0,22.0,1.0,0.0,2.0,2.0
5414,1,2019-01-07 14:30:00,9.0,23.0,1.0,1.0,0.0,1.0
5415,1,2019-01-07 14:40:00,11.0,21.0,1.0,0.0,2.0,2.0
5416,1,2019-01-07 14:50:00,10.0,22.0,1.0,1.0,0.0,1.0
5417,1,2019-01-07 15:00:00,10.0,22.0,1.0,0.0,0.0,0.0


In [48]:
#### Export

# CSV TEST
activite_full_resample.to_csv(ROOT_DATA_CLEAN + "time_serie_activity.csv", index=False)

In [45]:
activite_full_resample.to_hdf(ROOT_DATA_CLEAN + "time_serie_activity.h5", key="ts_activity")

In [49]:
## Lecture

In [50]:
# CSV TEST 12.4 secondes
%time df_read_test = pd.read_csv(ROOT_DATA_CLEAN + 'time_serie_activity.csv', parse_dates=['date'])

CPU times: user 11.5 s, sys: 832 ms, total: 12.4 s
Wall time: 12.4 s


In [51]:
%time df_read_test_H5 = pd.read_hdf(ROOT_DATA_CLEAN + 'time_serie_activity.h5', parse_dates=['date'])

CPU times: user 655 ms, sys: 292 ms, total: 947 ms
Wall time: 946 ms


#### Fonction de création

In [63]:
from vcub_keeper.create.creator import create_activity_time_series
from vcub_keeper.reader.reader import read_time_serie_activity

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
create_activity_time_series()

bordeaux-2020.csv
(11888595, 5)
bordeaux-2019.csv
(12527719, 5)
bordeaux-2018.csv
(1118425, 5)


In [65]:
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN)

In [66]:
ts_activity = ts_activity.collect()

In [67]:
# "available_stands": pl.Int8,
# "available_bikes": pl.Int8,
ts_activity.collect_schema()

Schema([('station_id', UInt8),
        ('date', Datetime(time_unit='ns', time_zone=None)),
        ('available_stands', Int8),
        ('available_bikes', Int8),
        ('status', UInt8),
        ('transactions_in', Int64),
        ('transactions_out', Int64),
        ('transactions_all', Int64)])

In [34]:
ts_activity.shape

(12794673, 8)

In [69]:
ts_activity.filter(pl.col("transactions_in") >= 255)

station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
u8,datetime[ns],i8,i8,u8,i64,i64,i64


In [70]:
ts_activity.tail()

station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
u8,datetime[ns],i8,i8,u8,i64,i64,i64
251,2020-08-28 11:10:00,28,12,1,0,0,0
251,2020-08-28 11:20:00,28,12,1,0,0,0
251,2020-08-28 11:30:00,26,14,1,2,0,2
251,2020-08-28 11:40:00,26,14,1,0,0,0
251,2020-08-28 11:50:00,26,14,1,0,0,0


## Industrialisation

In [1]:
from datetime import datetime

from vcub_keeper.config import ROOT_DATA_CLEAN
from vcub_keeper.create.creator import create_activity_time_series
from vcub_keeper.reader.reader import read_time_serie_activity

In [2]:
# Prend +/- 1 minute
create_activity_time_series()

bordeaux-2020.csv
(11888595, 5)
bordeaux-2019.csv
(12527719, 5)
bordeaux-2018.csv
(1118425, 5)


In [3]:
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN).collect()

In [6]:
# 27	2020-03-21 10:10:00
ts_activity.filter((pl.col("station_id") == 27) & (pl.col("date") == datetime(2020, 3, 21, 10, 10)))

station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
u8,datetime[ns],i8,i8,u8,i64,i64,i64
27,2020-03-21 10:10:00,11,9,1,0,1,1


In [7]:
print(ts_activity.tail())

shape: (5, 8)
┌────────────┬────────────┬────────────┬────────────┬────────┬────────────┬────────────┬───────────┐
│ station_id ┆ date       ┆ available_ ┆ available_ ┆ status ┆ transactio ┆ transactio ┆ transacti │
│ ---        ┆ ---        ┆ stands     ┆ bikes      ┆ ---    ┆ ns_in      ┆ ns_out     ┆ ons_all   │
│ u8         ┆ datetime[n ┆ ---        ┆ ---        ┆ u8     ┆ ---        ┆ ---        ┆ ---       │
│            ┆ s]         ┆ i8         ┆ i8         ┆        ┆ i64        ┆ i64        ┆ i64       │
╞════════════╪════════════╪════════════╪════════════╪════════╪════════════╪════════════╪═══════════╡
│ 251        ┆ 2020-08-28 ┆ 28         ┆ 12         ┆ 1      ┆ 0          ┆ 0          ┆ 0         │
│            ┆ 11:10:00   ┆            ┆            ┆        ┆            ┆            ┆           │
│ 251        ┆ 2020-08-28 ┆ 28         ┆ 12         ┆ 1      ┆ 0          ┆ 0          ┆ 0         │
│            ┆ 11:20:00   ┆            ┆            ┆        ┆            ┆  

In [8]:
ts_activity.shape

(12794673, 8)

## Analyse

In [76]:
import pandas as pd

from vcub_keeper.config import ROOT_DATA_CLEAN
from vcub_keeper.reader.reader import read_time_serie_activity
from vcub_keeper.visualisation import plot_station_activity

In [77]:
# Lecture de la time serie
ts_activity = read_time_serie_activity(path_directory=ROOT_DATA_CLEAN).collect()

In [78]:
ts_activity.shape

(12794673, 8)

In [79]:
ts_activity.select("date").describe()

statistic,date
str,str
"""count""","""12794673"""
"""null_count""","""0"""
"""mean""","""2019-11-22 08:33:53.762879"""
"""std""",null
"""min""","""2018-12-01 00:10:00"""
"""25%""","""2019-07-08 23:10:00"""
"""50%""","""2019-12-14 08:40:00"""
"""75%""","""2020-04-26 13:30:00"""
"""max""","""2020-08-28 11:50:00"""


### Absence de data dans la Time Series

In [ ]:
plot_station_activity(
    ts_activity.to_pandas(),
    station_id=2,
    features_to_plot=["available_bikes", "available_stands"],
    # start_date=start_date,
    # end_date=end_date,
    return_data=False,
)

Problème de time serie sans données le `2019-01-14 17:50` à `2019-02-21 10:10` pour la station N°2

In [75]:
file_name = "bordeaux-2019.csv"

column_dtypes = {"id": "uint8", "status": "category", "available_stands": "uint8", "available_bikes": "uint8"}
activite = pd.read_csv(ROOT_DATA_RAW + file_name, parse_dates=["timestamp"], dtype=column_dtypes)

activite.rename(columns={"id": "station_id"}, inplace=True)
activite.rename(columns={"timestamp": "date"}, inplace=True)

# Sorting DataFrame on station_id & date
activite = activite.sort_values(["station_id", "date"], ascending=[1, 1])

# Reset index
activite = activite.reset_index(drop=True)

In [76]:
activite.head()

,station_id,status,available_stands,available_bikes,date
0,1,open,18,14,2019-01-02 11:44:05
1,1,open,18,14,2019-01-02 11:49:03
2,1,open,18,14,2019-01-02 11:54:03
3,1,open,18,14,2019-01-02 11:59:02
4,1,open,18,14,2019-01-02 12:04:03


In [77]:
station_id = 2
date = "2019-02-21 11:00"
activite[(activite.station_id == station_id) & (activite.date <= date)].tail(15)

,station_id,status,available_stands,available_bikes,date
68120,2,open,9,9,2019-01-14 17:39:03
68121,2,open,10,8,2019-01-14 17:44:04
68122,2,open,11,7,2019-01-14 17:49:04
68123,2,open,5,15,2019-02-21 10:03:02
68124,2,open,5,15,2019-02-21 10:08:02
68125,2,open,5,15,2019-02-21 10:13:01
68126,2,open,5,15,2019-02-21 10:19:01
68127,2,open,5,15,2019-02-21 10:24:02
68128,2,open,5,15,2019-02-21 10:29:02
68129,2,open,5,15,2019-02-21 10:32:01


In [78]:
ts_activity[(ts_activity.station_id == station_id) & (ts_activity.date <= date)].tail(10)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
103519,2,2019-02-21 09:30:00,NaN,NaN,0.0,0.0,0.0,0.0
103520,2,2019-02-21 09:40:00,NaN,NaN,0.0,0.0,0.0,0.0
103521,2,2019-02-21 09:50:00,NaN,NaN,0.0,0.0,0.0,0.0
103522,2,2019-02-21 10:00:00,NaN,NaN,0.0,0.0,0.0,0.0
103523,2,2019-02-21 10:10:00,5.0,15.0,1.0,8.0,0.0,8.0
103524,2,2019-02-21 10:20:00,5.0,15.0,1.0,0.0,0.0,0.0
103525,2,2019-02-21 10:30:00,5.0,15.0,1.0,0.0,0.0,0.0
103526,2,2019-02-21 10:40:00,5.0,15.0,1.0,0.0,0.0,0.0
103527,2,2019-02-21 10:50:00,5.0,15.0,1.0,0.0,0.0,0.0
103528,2,2019-02-21 11:00:00,5.0,15.0,1.0,0.0,0.0,0.0


Il y a des trous des les données sources.

### Analyse des déconnexion des stations

Cf : https://github.com/armgilles/vcub_keeper/issues/21

In [80]:
# Lecture de la time serie
ts_activity = (
    read_time_serie_activity(path_directory=ROOT_DATA_CLEAN, post_pressessing_status=True).collect().to_pandas()
)
ts_activity_False = (
    read_time_serie_activity(path_directory=ROOT_DATA_CLEAN, post_pressessing_status=False).collect().to_pandas()
)

Si un point est déconnecté de façon orpheline, le `post_pressessing_status` à `True` le corrige (cf `2020-03-21 09:50:00`)

In [81]:
station_id = 27
date = "2020-03-21 09:30:00"
ts_activity[(ts_activity.station_id == station_id) & (ts_activity.date >= date)].head(7)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
1808252,27,2020-03-21 09:30:00,10,10,1,0,0,0
1808253,27,2020-03-21 09:40:00,10,10,1,0,0,0
1808254,27,2020-03-21 09:50:00,10,10,1,0,0,0
1808255,27,2020-03-21 10:00:00,10,10,1,0,0,0
1808256,27,2020-03-21 10:10:00,11,9,1,0,1,1
1808257,27,2020-03-21 10:20:00,11,9,1,0,0,0
1808258,27,2020-03-21 10:30:00,11,9,1,0,0,0


In [82]:
station_id = 27
date = "2020-03-21 09:30:00"
ts_activity_False[(ts_activity_False.station_id == station_id) & (ts_activity_False.date >= date)].head(7)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
1808252,27,2020-03-21 09:30:00,10,10,1,0,0,0
1808253,27,2020-03-21 09:40:00,10,10,1,0,0,0
1808254,27,2020-03-21 09:50:00,10,10,0,0,0,0
1808255,27,2020-03-21 10:00:00,10,10,1,0,0,0
1808256,27,2020-03-21 10:10:00,11,9,1,0,1,1
1808257,27,2020-03-21 10:20:00,11,9,1,0,0,0
1808258,27,2020-03-21 10:30:00,11,9,1,0,0,0


Si plusieurs points sont déconnectés à la suite, le `post_pressessing_status` à `True` ne change rien (à partir de 
`2019-04-01 06:10:00`)




In [83]:
station_id = 71
date = "2019-04-01 05:30:00"
ts_activity[(ts_activity.station_id == station_id) & (ts_activity.date >= date)].head(7)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
4866102,71,2019-04-01 05:30:00,13,5,1,0,0,0
4866103,71,2019-04-01 05:40:00,13,5,1,0,0,0
4866104,71,2019-04-01 05:50:00,18,0,1,0,5,5
4866105,71,2019-04-01 06:00:00,18,0,1,0,0,0
4866106,71,2019-04-01 06:10:00,18,0,0,0,0,0
4866107,71,2019-04-01 06:20:00,18,0,0,0,0,0
4866108,71,2019-04-01 06:30:00,18,0,0,0,0,0


In [84]:
station_id = 71
date = "2019-04-01 05:30:00"
ts_activity_False[(ts_activity_False.station_id == station_id) & (ts_activity_False.date >= date)].head(7)

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
4866102,71,2019-04-01 05:30:00,13,5,1,0,0,0
4866103,71,2019-04-01 05:40:00,13,5,1,0,0,0
4866104,71,2019-04-01 05:50:00,18,0,1,0,5,5
4866105,71,2019-04-01 06:00:00,18,0,1,0,0,0
4866106,71,2019-04-01 06:10:00,18,0,0,0,0,0
4866107,71,2019-04-01 06:20:00,18,0,0,0,0,0
4866108,71,2019-04-01 06:30:00,18,0,0,0,0,0
